## Initialise all helper matixes and select S-box

In [ ]:
from itertools import combinations
#box = (0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16)
box = (0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2)
#box = (0x4,0x0b,0x1f,0x14,0x1a,0x15,0x09,0x02,0x1b,0x05,0x08,0x12,0x1d,0x03,0x06,0x1c,0x1e,0x13,0x07,0x0e,0x00,0x0d,0x11,0x18,0x10,0x0c,0x01,0x19,0x16,0x0a,0x0f,0x17)
M, N = len(bin(max(box))[2:]), len(bin(max(box))[2:])
shiftM, shiftN = 1 << M, 1 << N
funs = [[] for _ in range(N)]
ll = [[0]*(shiftM) for _ in range(shiftN-1)]
wt = [[0]*(shiftM) for _ in range(shiftN-1)]
hamming = [0]*shiftM
ac = [[0]*(shiftM) for _ in range(shiftN-1)]

In [ ]:
### Split analysed S-box into boolean functions        
def splitFunctions(box):
    for num in box:
        for fun in funs:
            fun.append(num & 1)
            num = num >> 1 
    for i, fun in enumerate(funs):
        print(f'Function {i}: {fun}')

### Calculate hamming weights for all input values
def hammingWeights():
    for i in range(shiftM):
        hamming[i] = bin(i).count('1')

### Calculate all linear combinations of S-box functions
def linearCombo():
    if(N == 1):
        for i in range(shiftM):
            ll[0][i] = funs[0][i]
    else:
        for i in range(1, shiftN):
            for j in range(N):
                if(i >> j & 0x1):
                    for k in range(shiftM):
                        ll[i-1][k] = ll[i-1][k] ^ funs[j][k]
  
### Compute fast Walsh transform of of functions                        
def walsh():
    for k in range(shiftN-1):
        for i in range(shiftM):
            wt[k][i] = 1 if(ll[k][i] == 0) else -1
        for i in range(1,M+1):
            m = 1 << i
            for j in range(0,shiftM,m):
                t1 = j
                t2 = j + m//2
                for _ in range(m//2):
                    a = wt[k][t1]
                    b = wt[k][t2]
                    wt[k][t1] = a + b
                    wt[k][t2] = a - b
                    t1 += 1
                    t2 += 1

### Compute Autocorrelation                
def autocorrelation():
    for z in range(shiftN - 1):
        for i in range(shiftM):
            ac[z][i] = -1 * wt[z][i] * wt[z][i]
        for i in range(1,M+1):
            m = 1 << i
            halfm = m//2
            for r in range(0,shiftM,m):
                t1 = r
                t2 = r+halfm
                for _ in range(halfm):
                    a = ac[z][t1]
                    b = ac[z][t2]
                    ac[z][t1] = a + b
                    ac[z][t2] = a - b
                    t1 += 1
                    t2 += 1
        for i in range(shiftM):
            ac[z][i] //= (1 << M)*(-1)  

In [ ]:
### Find the smallest number with n bits set
def minSet(n):
    if(n <= 0):
        return 0
    num = 0
    for _ in range(n):
        num |= 1
        num <<= 1
    num >>= 1
    return num

### Generate the next mask to be used for SAC
### The next mask is the next number with the same number of bits set
def nextMask(mask):
    if(mask == 0):
        return 0
    smallest = mask & (-mask)
    ripple = mask + smallest
    newSmallest = ripple & (-ripple)
    ones = ((newSmallest//smallest) >> 1) - 1
    return ripple | ones

### Calculate SAC for a single function using a masking method
def sac(fun, funNum, order):
    sacVal = 0
    mask = minSet(order)
    for _ in range(funNum):
        for val in range(len(fun)):
            sacVal += fun[val] ^ fun[val ^ mask]
        mask = nextMask(mask)
    return sacVal

### Calculate SAC for all functions and their combinations
def SACCheck(order):
    if(order >= M):
        print('Order too high for given S-box')
        return None
    ind = tuple(range(len(funs)))
    num = 0
    den = 0
    for i in range(1,len(funs)+1):
        for sub in combinations(ind, i):
            fun = funs[sub[0]]
            for f in sub[1:]:
                for k, bit in enumerate(funs[f]):
                    fun[k] ^= bit
            num += sac(fun, len(funs), order) / len(funs[0])
            den += 1
    return round(num/den, 2)
 
### Calculate nonlinearity of a function based on Hamming distance         
def nonlinearity(fun):
    distSum = 9999999
    for i in range(len(fun)):
        bitSum = 0
        bitSumNeg = 0
        for j in range(len(fun)):
            n = i & j
            bit = 0
            while n:
                n &= n - 1
                bit ^= 1
            bitSum += fun[j] ^ bit
            bitSumNeg += fun[j] ^ (not bit)
        distSum = min(distSum, bitSum, bitSumNeg)
    return distSum

### Find the nonlinearity of a set of functions       
def nonlinearCheck():
    ind = tuple(range(len(funs)))
    nonl = 9999999
    for i in range(1,len(funs)+1):
        for sub in combinations(ind, i):
            fun = funs[sub[0]]
            for f in sub[1:]:
                for k, bit in enumerate(funs[f]):
                    fun[k] ^= bit
            nonl = min(nonl, nonlinearity(fun))
    return nonl
  
### Check balancedness of functions    
def balancedness():
    for fun in funs:
        if(fun.count(0) != fun.count(1)):
            return(False)
        return True
                
### Calculate Correlation Immunity (CI)             
def corrIm():
    col = shiftN - 1
    row = shiftM
    minCor = 99999999
    cor = 0
    i = 0
    for i in range(col):
        order = 1
        j = 1
        while j < row:
            if(order == hamming[j] and wt[i][j] != 0):
                cor = order - 1
                break
            if(j == (row-1) and order <= M):
                j = 1
                order += 1
            cor = order - 2
            j += 1
        minCor = min(minCor, cor)
    return minCor
 
### Calculate Absolute Indicator (AI)     
def absoluteIndicator():
    max, tmp, tmp2 = 0, 0, 0
    for j in range(shiftN-1):
        tmp = abs(ac[j][1])
        for i in range(2,shiftM):
            tmp2 = abs(ac[j][i])
            if(tmp2 > tmp):
                tmp = tmp2
        if(tmp > max):
            max = tmp
    return max

### Calculate Sum-ofsquares Indicator (SSI)
def sumOfSquare():
    max, sum = 0, 0
    for j in range(shiftN-1):
        for i in range(shiftM):
            sum += ac[j][i] * ac[j][i]
        if(sum > max):
            max = sum
        sum = 0
    return max

## Prepare all matrixes

In [ ]:
### Remeber to reinitialise all matrixes after each run
splitFunctions(box) 
hammingWeights()
linearCombo()
walsh()
autocorrelation()

## Calculate metrics

In [ ]:
for i in range(1, M):
    print(f'SAC {i}: {SACCheck(i)}')
print(f'Nonlinearity: {nonlinearCheck()}')
print(f'Balancedness: {balancedness()}')
print(f'Correlation Immunity: {corrIm()}')
print(f'Absolute Indicator {absoluteIndicator()}')
print(f'Sum-of-squares Indicator: {sumOfSquare()}')